In [1]:
text=open('warpeace_input.txt','r').read()

In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Flatten,BatchNormalization,Dropout 

In [3]:
text[:100]

'\ufeff"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, i'

In [4]:
chars = list(set(text))
VOCAB_SIZE = len(chars)

In [5]:
char2idx = {u:i for i, u in enumerate(chars)}
idx2char = np.array(chars)
print(char2idx)

{"'": 0, 'M': 1, 'a': 2, 'g': 3, 'D': 4, 'z': 5, 'B': 6, 'P': 7, 'J': 8, 'f': 9, 'F': 10, 'Q': 11, '6': 12, '\n': 13, 'O': 14, 'Z': 15, 'v': 16, '8': 17, 'Y': 18, 'ê': 19, 'I': 20, 'j': 21, '.': 22, '(': 23, 'y': 24, 'T': 25, '!': 26, 'q': 27, 'K': 28, '2': 29, 'x': 30, 's': 31, '"': 32, 'V': 33, '*': 34, ')': 35, 'E': 36, 'X': 37, '3': 38, '-': 39, 'o': 40, 't': 41, 'W': 42, '9': 43, 'n': 44, 'r': 45, 'à': 46, 'i': 47, 'L': 48, 'G': 49, 'b': 50, ':': 51, 'N': 52, 'h': 53, 'm': 54, '4': 55, '=': 56, 'é': 57, 'S': 58, 'R': 59, 'C': 60, 'l': 61, 'd': 62, '1': 63, 'ä': 64, 'u': 65, 'w': 66, '?': 67, 'p': 68, 'k': 69, '/': 70, '0': 71, 'e': 72, '7': 73, ' ': 74, 'c': 75, '\ufeff': 76, ';': 77, 'U': 78, '5': 79, 'A': 80, 'H': 81, ',': 82}


In [6]:
print(idx2char)

["'" 'M' 'a' 'g' 'D' 'z' 'B' 'P' 'J' 'f' 'F' 'Q' '6' '\n' 'O' 'Z' 'v' '8'
 'Y' 'ê' 'I' 'j' '.' '(' 'y' 'T' '!' 'q' 'K' '2' 'x' 's' '"' 'V' '*' ')'
 'E' 'X' '3' '-' 'o' 't' 'W' '9' 'n' 'r' 'à' 'i' 'L' 'G' 'b' ':' 'N' 'h'
 'm' '4' '=' 'é' 'S' 'R' 'C' 'l' 'd' '1' 'ä' 'u' 'w' '?' 'p' 'k' '/' '0'
 'e' '7' ' ' 'c' '\ufeff' ';' 'U' '5' 'A' 'H' ',']


In [7]:
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print(text_as_int)

[76 32 42 ... 65 31 22]


In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\ufeff"Well, Princ' ---- characters mapped to int ---- > [76 32 42 72 61 61 82 74  7 45 47 44 75]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
no_samples = len(text)/seq_length

x=[]
y=[]

for z in range(int(len(text)/seq_length)):
    for i in range(seq_length):
        xe=[]
        for j in range(seq_length-i-1):
            xe.append(char2idx[' '])
        for j in range(i+1):
            xe.append(text_as_int[z*seq_length+j])
        xe=np.array(xe)

        x.append(xe)
        y.append(text_as_int[(z)*seq_length+i+1])
 

In [11]:
model=Sequential()
model.add(Embedding(VOCAB_SIZE,256,input_length=seq_length))
model.add(LSTM(1000,return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(200,activation='relu'))
model.add(Dense(VOCAB_SIZE,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
temp_y=np.zeros(shape=(len(y),VOCAB_SIZE))
x=np.array(x)
print(temp_y.shape)
print(x.shape)
for i in range(len(y)):
    temp_y[i][y[i]]=1
y=temp_y

(3196200, 83)
(3196200, 100)


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.2,test_size=0.02,random_state=42)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(639240, 100)
(639240, 83)
(63924, 100)
(63924, 83)


In [17]:
import tensorflow as tf
import os
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [18]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=3,batch_size=100,callbacks=[cp_callback])

Epoch 1/3
6393/6393 [==============================] - ETA: 0s - loss: 2.1508 - accuracy: 0.3805
Epoch 00001: saving model to cp.ckpt
6393/6393 [==============================] - 8637s 1s/step - loss: 2.1508 - accuracy: 0.3805 - val_loss: 1.9214 - val_accuracy: 0.4353
Epoch 2/3
6393/6393 [==============================] - ETA: 0s - loss: 1.8133 - accuracy: 0.4627
Epoch 00002: saving model to cp.ckpt
6393/6393 [==============================] - 8696s 1s/step - loss: 1.8133 - accuracy: 0.4627 - val_loss: 1.7794 - val_accuracy: 0.4766
Epoch 3/3
6393/6393 [==============================] - ETA: 0s - loss: 1.6515 - accuracy: 0.5054
Epoch 00003: saving model to cp.ckpt
6393/6393 [==============================] - 9052s 1s/step - loss: 1.6515 - accuracy: 0.5054 - val_loss: 1.7239 - val_accuracy: 0.4963


In [21]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=3,batch_size=100,callbacks=[cp_callback])

Epoch 1/3
6393/6393 [==============================] - ETA: 0s - loss: 1.5352 - accuracy: 0.5366
Epoch 00001: saving model to cp.ckpt
6393/6393 [==============================] - 8909s 1s/step - loss: 1.5352 - accuracy: 0.5366 - val_loss: 1.7104 - val_accuracy: 0.5041
Epoch 2/3
6393/6393 [==============================] - ETA: 0s - loss: 1.4428 - accuracy: 0.5622
Epoch 00002: saving model to cp.ckpt
6393/6393 [==============================] - 8960s 1s/step - loss: 1.4428 - accuracy: 0.5622 - val_loss: 1.8284 - val_accuracy: 0.5055
Epoch 3/3
  32/6393 [..............................] - ETA: 2:14:29 - loss: 1.3310 - accuracy: 0.6044

KeyboardInterrupt: 

In [19]:
def predict(txt, word_gen):
    inp_len = len(txt) 
    temp_txt=str()
    if inp_len<100:
        
        for i in range(seq_length-len(txt)):
            temp_txt+=' '
        temp_txt += txt   
    else:
        temp_txt = txt[:100]

    int_txt=[char2idx[c] for c in temp_txt ]
    
    int_txt= np.array(int_txt)
    int_txt = np.reshape(int_txt,(1,100))

    
    for i in range(word_gen):
        
        pred = model.predict(int_txt)
        
        txt+=idx2char[np.argmax(pred)]
        int_txt[0][:99]=int_txt[0][1:]
        int_txt[0][99]=np.argmax(pred)

        
    return txt


In [25]:
print(predict('war is god',200))

war is god the soldiers and the same thought the something to the
more than there words the going to a moment.

"Why would sat the strong the mon there wounded to the the thousht the
 atk the that the thought t
